In [1]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

In [14]:
# Import the datasets
pp = pd.read_csv('data/Project Plot Points.csv',delimiter=';')
cp = pd.read_csv('data/Control Plot Points.csv',delimiter=';')

In [ ]:
# Drop X and Y (confidential info, can't publictlt shared)
pp.drop(['X', 'Y'], axis=1, inplace=True)
cp.drop(['X', 'Y'], axis=1, inplace=True)

In [25]:
print(pp.columns)
print(cp.columns)

Index(['PP ID', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022',
       '2023', '2024', 'Euclidean Distance'],
      dtype='object')
Index(['CP ID', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022',
       '2023', '2024', 'Euclidean Distance', 'Slope'],
      dtype='object')


In [26]:
display(cp.head(), pp.head())

,CP ID,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,Euclidean Distance,Slope
0,49830,0.1289,0.0695,0.1128,0.2049,0.2838,0.3574,0.4418,0.4829,0.5190,0.5438,25031.2070,0.0577
1,49829,0.0924,0.0762,0.1537,0.2143,0.2982,0.3917,0.4649,0.5011,0.5141,0.5161,25040.3633,0.0573
2,49749,0.0639,0.0683,0.1097,0.1513,0.1995,0.2807,0.3669,0.4154,0.4714,0.4905,24974.9961,0.0540
3,49908,0.1097,0.0933,0.1106,0.2302,0.3026,0.3634,0.4214,0.4507,0.4752,0.4824,25096.7031,0.0507
4,49750,0.0555,0.0693,0.0872,0.1391,0.1888,0.2621,0.3436,0.3914,0.4375,0.4446,24966.0000,0.0502


,PP ID,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,Euclidean Distance
0,18482,0.2823,0.2644,0.2645,0.2848,0.2947,0.2964,0.3194,0.3060,0.3162,0.3134,6058.0869
1,39716,0.2429,0.2958,0.3218,0.3063,0.2996,0.3215,0.3507,0.3281,0.3195,0.2899,9544.7607
2,27158,0.3623,0.3631,0.3977,0.4050,0.3964,0.4171,0.4505,0.4012,0.4051,0.4070,5702.9170
3,39583,0.2815,0.2844,0.3425,0.3100,0.2451,0.3462,0.3781,0.3883,0.4212,0.4259,5409.5381
4,25567,0.3042,0.3017,0.3583,0.3687,0.3549,0.3888,0.3858,0.4036,0.3738,0.3690,6769.5317


In [17]:
# Celan NaN in cp
cp.dropna(inplace=True)

In [ ]:
# Function to find 3 nearest neighbors for each labeled data point
def find_knn_for_labels(df_labeled, df_unlabeled, k=3):
    required_columns = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024'] #+'EuclideanDistance'

    # Ensure required columns are present in both DataFrames
    for col in required_columns:
        if col not in df_labeled.columns:
            raise KeyError(f"Column '{col}' not found in labeled data.")
        if col not in df_unlabeled.columns:
            raise KeyError(f"Column '{col}' not found in unlabeled data.")

    results = []

    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(df_unlabeled[required_columns])

    for index, row in df_labeled.iterrows():
        label = row['PP ID']
        data_point = row[required_columns].values.reshape(1, -1)

        # Debug statement to check the data point
        print(f"Processing label {label} with data point: {data_point}")

        distances, indices = knn.kneighbors(data_point)
        nearest_neighbors = df_unlabeled.iloc[indices[0]].reset_index(drop=True)

        # Add label to the neighbors DataFrame
        nearest_neighbors['PP ID'] = label
        nearest_neighbors['distance'] = distances[0]

        results.append(nearest_neighbors)

    return pd.concat(results).reset_index(drop=True)

In [27]:
# Finding 3 nearest neighbors for each labeled data point
knn_results = find_knn_for_labels(pp, cp, k=1)

# Displaying the results
# for label, neighbors in knn_results.items():
#     print(f"Label: {label}")
#     print(neighbors)
#     print()

Processing label 18482.0 with data point: [[2.8230000e-01 2.6440000e-01 2.6450000e-01 2.8480000e-01 2.9470000e-01
  2.9640000e-01 3.1940000e-01 3.0600000e-01 3.1620000e-01 3.1340000e-01
  6.0580869e+03]]
Processing label 39716.0 with data point: [[2.4290000e-01 2.9580000e-01 3.2180000e-01 3.0630000e-01 2.9960000e-01
  3.2150000e-01 3.5070000e-01 3.2810000e-01 3.1950000e-01 2.8990000e-01
  9.5447607e+03]]
Processing label 27158.0 with data point: [[3.623000e-01 3.631000e-01 3.977000e-01 4.050000e-01 3.964000e-01
  4.171000e-01 4.505000e-01 4.012000e-01 4.051000e-01 4.070000e-01
  5.702917e+03]]
Processing label 39583.0 with data point: [[2.8150000e-01 2.8440000e-01 3.4250000e-01 3.1000000e-01 2.4510000e-01
  3.4620000e-01 3.7810000e-01 3.8830000e-01 4.2120000e-01 4.2590000e-01
  5.4095381e+03]]
Processing label 25567.0 with data point: [[3.0420000e-01 3.0170000e-01 3.5830000e-01 3.6870000e-01 3.5490000e-01
  3.8880000e-01 3.8580000e-01 4.0360000e-01 3.7380000e-01 3.6900000e-01
  6.76953

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does 

In [28]:
knn_results

,CP ID,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,Euclidean Distance,Slope,PP ID,distance
0,372175,0.2591,0.2735,0.2870,0.3106,0.2937,0.3055,0.3032,0.3003,0.2934,0.2940,6058.0869,0.0031,18482.0,0.055388
1,141592,0.3076,0.3098,0.3125,0.3050,0.3130,0.3007,0.3010,0.3082,0.3071,0.3069,9544.7607,-0.0004,39716.0,0.091604
2,225944,0.3226,0.3681,0.3977,0.3773,0.3767,0.3630,0.4027,0.4119,0.3973,0.3750,5702.9170,0.0049,27158.0,0.095747
3,280644,0.2969,0.3048,0.3493,0.3365,0.3186,0.3293,0.3427,0.3461,0.3351,0.3079,5409.5381,0.0020,39583.0,0.177372
4,226733,0.3446,0.3536,0.4015,0.3956,0.3776,0.3867,0.3931,0.3880,0.3796,0.3799,6769.5317,0.0026,25567.0,0.088794
5,373239,0.1376,0.1662,0.2382,0.2254,0.2296,0.2462,0.1920,0.3060,0.2679,0.2559,8727.2920,0.0123,7052.0,0.152191
6,199302,0.2890,0.2976,0.3129,0.2965,0.2984,0.3099,0.3253,0.3183,0.3060,0.3174,8773.6328,0.0027,19943.0,0.084253
7,231711,0.3126,0.3222,0.3370,0.3071,0.3089,0.3126,0.3285,0.3275,0.3442,0.3415,12597.6914,0.0026,5324.0,0.231703
8,230321,0.3237,0.3408,0.3683,0.3623,0.3653,0.3612,0.3515,0.3482,0.3549,0.3703,11167.9199,0.0023,41670.0,0.089225
9,203165,0.2231,0.2339,0.2736,0.2648,0.2694,0.2784,0.2928,0.3011,0.2997,0.3016,12585.6143,0.0085,23086.0,0.168989


In [ ]:
knn_results['Slope'].mean()

0.0045766666666666655